beispielhaft synthetische Daten

In [1]:
import pandas as pd
import numpy as np

# Beispiel: 1500 Stunden synthetische Daten (ca. 2 Monate)
np.random.seed(42)
n_hours = 1500
dates = pd.date_range("2020-01-01", periods=n_hours, freq="h")

# Exogene Daten simulieren (z.B. Wind)
hours = np.arange(n_hours)
wind_speed = 10 + 5*np.sin(2*np.pi*hours/24) + np.random.normal(0, 2, n_hours)  # tägliches Muster + Rauschen
wind_dir   = (180 + 45*np.sin(2*np.pi*hours/24 + np.pi/4) + np.random.normal(0, 20, n_hours)) % 360  # Richtung in Grad

# Wasserstand simulieren (tägliche + wöchentliche Schwankung + Wind-Einfluss + Rauschen)
water_level = (
    50 
    + 5*np.sin(2*np.pi*hours/24)    # tägliche Welle (Amplitude 5)
    + 3*np.sin(2*np.pi*hours/168)   # wöchentliche Welle (Amplitude 3)
    + 0.1 * wind_speed             # angenommener positiver Einfluss des Winds
    + np.random.normal(0, 1, n_hours)  # Messrauschen
)

# DataFrame mit Basiszeitreihen
df = pd.DataFrame({
    'water_level': water_level,
    'wind_speed': wind_speed,
    'wind_dir': wind_dir
}, index=dates)

# Beispielhafte Betrachtung der ersten Zeilen
print(df.head(3))


                     water_level  wind_speed    wind_dir
2020-01-01 00:00:00    49.191535   10.993428  227.387027
2020-01-01 01:00:00    51.647640   11.017567  207.947429
2020-01-01 02:00:00    53.690122   13.795377  207.102685


Als Nächstes erzeugen wir die Lag-Features.

In [3]:
# ignore PerformanceWarning
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Feature-Lags definieren
lags_target = [1, 24, 168]           # Lags für Wasserstand (1h, 24h, 168h zurück)
lags_exog   = [24, 48, 72]           # Lags für exogene Merkmale (24h, 48h, 72h zurück)

# DataFrame für Features kopieren
df_feat = df.copy()

# Ziel-Lags (Wasserstand) hinzufügen
for lag in lags_target:
    df_feat[f'water_lag{lag}'] = df_feat['water_level'].shift(lag)

# Exogene Lags hinzufügen
for lag in lags_exog:
    df_feat[f'wind_speed_lag{lag}'] = df_feat['wind_speed'].shift(lag)
    df_feat[f'wind_dir_lag{lag}']   = df_feat['wind_dir'].shift(lag)

# Zukünftige Zielwerte als separate Spalten (t+1 bis t+168) zur Vorbereitung (nur für Training)
H = 168  # Horizont
for h in range(1, H+1):
    df_feat[f'target_t+{h}'] = df_feat['water_level'].shift(-h)

# Fehlende Werte durch Shifts entfernen
df_feat = df_feat.dropna()
print(df_feat.shape)
print(df_feat.columns.tolist()[:10], "...", df_feat.columns.tolist()[-3:])


(1164, 180)
['water_level', 'wind_speed', 'wind_dir', 'water_lag1', 'water_lag24', 'water_lag168', 'wind_speed_lag24', 'wind_dir_lag24', 'wind_speed_lag48', 'wind_dir_lag48'] ... ['target_t+166', 'target_t+167', 'target_t+168']


In [5]:
# Beispiel: erste 5 Feature-Spalten und ersten 3 Zeilen anzeigen
cols_to_show = ['water_level', 'water_lag1', 'water_lag24', 'wind_speed_lag24', 'wind_speed_lag48']
print(df_feat[cols_to_show].head(3))
print("\n...", df_feat[[f'target_t+1', f'target_t+168']].head(3))


                     water_level  water_lag1  water_lag24  wind_speed_lag24  \
2020-01-08 00:00:00    50.854299   50.001734    49.138745         10.519766   
2020-01-08 01:00:00    51.968964   50.854299    50.775998         12.857741   
2020-01-08 02:00:00    52.850541   51.968964    50.595828         10.026099   

                     wind_speed_lag48  
2020-01-08 00:00:00         11.582064  
2020-01-08 01:00:00          9.475320  
2020-01-08 02:00:00         15.305589  

...                      target_t+1  target_t+168
2020-01-08 00:00:00   51.968964     49.469076
2020-01-08 01:00:00   52.850541     52.322432
2020-01-08 02:00:00   55.670065     54.559561


Konstruktion der Trainings-Matrizen X und Y (Multi-Output-Format)

Nun trennen wir die vorbereiteten Daten in die Feature-Matrix X (Eingabedaten) und die Zielmatrix Y (gleichzeitige Ausgabe von 168 Werten):

    X enthält pro Zeitschritt alle Feature-Spalten (aktuelle und verzögerte Werte, ohne die zukünftigen Zielspalten).

    Y enthält pro Zeitschritt die 168 Zielwerte in einem Vektor. Das heißt, für jede Zeile gibt es 168 gleichzeitige Outputs – in unserem Beispiel also den Wasserstand 1h voraus bis 168h voraus.

In [6]:
# Feature- und Ziel-Spalten definieren
feature_cols = ['water_level'] + [f'water_lag{lag}' for lag in lags_target] \
               + [f'wind_speed_lag{lag}' for lag in lags_exog] \
               + [f'wind_dir_lag{lag}' for lag in lags_exog]
target_cols = [f'target_t+{h}' for h in range(1, H+1)]

# Aufteilen in X und Y
X = df_feat[feature_cols].values
Y = df_feat[target_cols].values
print("X Shape:", X.shape, "| Y Shape:", Y.shape)


X Shape: (1164, 10) | Y Shape: (1164, 168)


Training des XGBoost Multi-Output-Regressors

In [7]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor

# XGBoost Basismodell definieren
xgb_base = XGBRegressor(n_estimators=50, max_depth=3, learning_rate=0.1, random_state=42)
# Multi-Output Wrapper
model = MultiOutputRegressor(xgb_base, n_jobs=-1)

# Training (Fit auf X und Y)
model.fit(X, Y)


Wir sehen, dass len(model.estimators_) = 168 sein sollte. Jeder dieser Estimatoren ist ein trainiertes XGBRegressor-Objekt. Die Feature-Importance-Ausgabe (falls aktiviert) gibt Hinweise, welche Eingangsmerkmale für die 1-Stunden-Prognose besonders wichtig waren (z. B. wahrscheinlich der unmittelbar vorige Wasserstand und ggf. Wind 24h zuvor, je nach Daten).

In [8]:
print("Anzahl Teilmodelle:", len(model.estimators_))
print("Feature Importance des Modells für t+1:", model.estimators_[0].feature_importances_)


Anzahl Teilmodelle: 168
Feature Importance des Modells für t+1: [0.3731011  0.00751804 0.00658373 0.52786744 0.00550194 0.00448346
 0.00410286 0.02747366 0.02302146 0.02034628]


4. Vorhersagen erzeugen und Prognose einrollen (Rolling Forecast)

In [10]:
# Beispiel: Aufteilen in Train und Test (letzte 200 Stunden als Test)
train_df = df.iloc[:-200]
test_df  = df.iloc[-200:]

# Unser Modell wurde auf train_df trainiert (siehe oben).
# Wir führen nun eine Rolling Forecast auf dem Testzeitraum durch.

import numpy as np

history = train_df.copy()  # Kopie der Historie, die wir schrittweise erweitern
predictions = []

for i in range(len(test_df)):
    # Nächster Zeitpunkt, für den wir vorhersagen (Basispunkt ist die letzte bekannte Stunde in 'history')
    current_time = history.index[-1]
    # Feature-Vektor für den aktuellen Basispunkt erstellen
    feat_vec = []
    feat_vec.append(history.iloc[-1]['water_level'])        # aktueller Wasserstand (t=0)
    feat_vec.append(history.iloc[-2]['water_level'])        # Wasserstand t-1
    feat_vec.append(history.iloc[-25]['water_level'])       # Wasserstand t-24
    feat_vec.append(history.iloc[-169]['water_level'])      # Wasserstand t-168
    feat_vec.append(history.iloc[-25]['wind_speed'])        # Wind t-24
    feat_vec.append(history.iloc[-49]['wind_speed'])        # Wind t-48
    feat_vec.append(history.iloc[-73]['wind_speed'])        # Wind t-72
    feat_vec.append(history.iloc[-25]['wind_dir'])          # Windrichtung t-24
    feat_vec.append(history.iloc[-49]['wind_dir'])          # Windrichtung t-48
    feat_vec.append(history.iloc[-73]['wind_dir'])          # Windrichtung t-72
    feat_vec = np.array(feat_vec).reshape(1, -1)

    # Multi-Output-Vorhersage für die nächsten 168h
    y_pred = model.predict(feat_vec)[0]  # Vorhersageergebnis als Array der Länge 168
    
    # Speichere z.B. die 1-Stunden-Vorhersage aus diesem Durchlauf
    predictions.append(y_pred[0])
    
    # "Neue Stunde kommt an": Füge den tatsächlichen nächsten Messwert der History hinzu (aus test_df)
    next_index = test_df.index[i]
    next_data = test_df.iloc[i]  # tatsächlicher Wert zum nächsten Zeitpunkt
    next_data_df = next_data.to_frame().T
    next_data_df.index = [next_index]
    history = pd.concat([history, next_data_df])


In [11]:
predictions

[53.445244,
 53.651276,
 53.738083,
 54.153603,
 53.764694,
 52.858063,
 50.413673,
 50.070763,
 48.467304,
 47.103855,
 46.544094,
 44.65053,
 43.793278,
 43.65622,
 43.38442,
 43.444466,
 43.538662,
 43.715225,
 45.107098,
 46.364685,
 48.530045,
 49.725697,
 50.991974,
 53.956963,
 53.628094,
 54.343002,
 54.646717,
 54.335224,
 54.55131,
 54.355816,
 51.922718,
 50.53493,
 48.858162,
 49.423237,
 47.958164,
 46.988285,
 44.447334,
 45.58905,
 44.21963,
 44.22204,
 45.233356,
 46.77721,
 47.13565,
 50.055416,
 50.545227,
 52.1785,
 53.288773,
 54.92794,
 54.268078,
 56.18881,
 56.9338,
 56.483383,
 56.277714,
 56.66338,
 54.07577,
 54.80619,
 51.04761,
 51.77057,
 49.355865,
 48.913803,
 47.667377,
 48.662117,
 46.605755,
 47.27766,
 47.582256,
 49.218563,
 49.03837,
 50.73194,
 51.397892,
 54.391094,
 54.08673,
 55.30302,
 56.65819,
 58.331215,
 59.011337,
 59.18639,
 56.947502,
 55.99378,
 56.671944,
 55.78597,
 52.93164,
 52.45617,
 50.242855,
 49.88981,
 48.967728,
 49.898853,
 